In [1]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau

# Define transformations
data_transform_train = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop((224, 224)),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

data_transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

training_data = datasets.Flowers102(
    root="data",
    split="train",
    download=True,
    transform=data_transform_train
)

test_data = datasets.Flowers102(
    root="data",
    split="test",
    download=True,
    transform=data_transform_test
)

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64)

In [2]:
import torch.nn as nn
import torch.nn.functional as F
# Get cpu, gpu or mps device for training.

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1, padding=2)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, 1, padding=2)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, 1, padding=2)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, 1, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(50176, 1024)
        self.bn5 = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 512)
        self.bn6 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 256)
        self.bn7 = nn.BatchNorm1d(256)
        self.fc4 = nn.Linear(256, 102)
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = torch.flatten(x, 1)
        x = F.relu(self.bn5(self.fc1(x)))
        #x = self.dropout(x)
        x = F.relu(self.bn6(self.fc2(x)))
        #x = self.dropout(x)
        x = F.relu(self.bn7(self.fc3(x)))
        x = self.fc4(x)
        return self.logsoftmax(x)

model = NeuralNetwork()

learning_rate = 1e-3
batch_size = 64
epochs = 100

In [4]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min',factor=0.5, patience=5)

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
 
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)  # move data to device
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 1 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
 
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)  # move data to device
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return test_loss

In [ ]:
import time

loss_fn = nn.CrossEntropyLoss()

# Define early stopping parameters
patience = 10
min_delta = 0.001

best_loss = float('inf')
counter = 0

# Timer
start_time = time.time()

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    validation_loss = test_loop(test_dataloader, model, loss_fn)
    scheduler.step(validation_loss)

    # Early stopping logic
    if validation_loss < best_loss - min_delta:
        best_loss = validation_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered.")
            break
            
end_time = round((time.time()-start_time)/60, 3)

print("Done! This took", end_time, "minutes")


Epoch 1
-------------------------------
loss: 4.715024  [   64/ 1020]
loss: 4.688758  [  128/ 1020]
loss: 4.441965  [  192/ 1020]
loss: 4.581042  [  256/ 1020]
loss: 4.405237  [  320/ 1020]
loss: 4.573836  [  384/ 1020]
loss: 4.371413  [  448/ 1020]
loss: 4.400090  [  512/ 1020]
loss: 4.223102  [  576/ 1020]
loss: 4.233042  [  640/ 1020]
loss: 4.366311  [  704/ 1020]
loss: 4.193057  [  768/ 1020]
loss: 4.011764  [  832/ 1020]
loss: 4.208547  [  896/ 1020]
loss: 4.239359  [  960/ 1020]
loss: 4.053277  [ 1020/ 1020]
Test Error: 
 Accuracy: 5.1%, Avg loss: 4.329369 

Epoch 2
-------------------------------
loss: 3.760036  [   64/ 1020]
loss: 3.864957  [  128/ 1020]
loss: 3.882756  [  192/ 1020]
loss: 3.743947  [  256/ 1020]
loss: 3.687417  [  320/ 1020]
loss: 3.691432  [  384/ 1020]
loss: 3.796304  [  448/ 1020]
loss: 3.525075  [  512/ 1020]
loss: 3.517890  [  576/ 1020]
loss: 3.618374  [  640/ 1020]
loss: 3.472885  [  704/ 1020]
loss: 3.585022  [  768/ 1020]
loss: 3.414358  [  832/ 1020]